In [38]:
import sys
import os
import logging
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
import math 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix 

from collections import defaultdict
from collections import Counter
path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle/"
sub_path = "../../data/tencent2020/sub/"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [29]:
user_ids = pd.read_pickle(f"{pickle_path}/user_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_ids = user_ids.merge(user, how='left', on='user_id')
del user
user_ids.head()

,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len,age,gender
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[0, 136, 0, 0, 1334, 0, 0, 0, 0, 1454, 64, 64,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 0, 326, 73, 217, 64, 238, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]",12,12,6,3,12,9,4.0,1.0
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...","[58788, 139702, 38066, 541125, 14495, 392680, ...","[87, 80, 129, 129, 1400, 87, 0, 1261, 0, 0, 11...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[22885, 10686, 18562, 25932, 768, 22885, 34505...","[318, 238, 6, 6, 317, 318, 47, 6, 47, 47, 242,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",42,42,20,3,36,15,10.0,1.0
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...","[586668, 713448, 629278, 728308, 527601, 64310...","[36256, 40905, 1674, 35985, 1674, 0, 0, 1031, ...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[32974, 9877, 18492, 14186, 17018, 9058, 8371,...","[0, 0, 322, 0, 322, 6, 54, 6, 322, 322, 205, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",30,30,17,6,28,8,7.0,2.0
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...","[37966, 524312, 511235, 1638619, 1698206, 1953...","[1862, 0, 2625, 38743, 41265, 37758, 39904, 39...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[19451, 7976, 13084, 12130, 23664, 811, 10172,...","[238, 25, 248, 0, 0, 0, 0, 0, 88, 319, 238, 6,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",29,29,18,6,26,10,5.0,1.0
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[265971, 314795, 24966, 41148, 751113, 925792,...","[0, 0, 87, 136, 0, 136, 0, 0, 0, 1064, 0, 2620...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[11882, 992, 22885, 9706, 38760, 2862, 17745, ...","[297, 0, 318, 6, 322, 6, 288, 322, 319, 238, 3...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",33,33,7,4,30,18,4.0,1.0


In [30]:
creative_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/creative_id_w2v100.bin", binary=True)
ad_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/ad_id_w2v100.bin", binary=True)
advertiser_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/advertiser_id_w2v50.bin", binary=True)
product_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/product_id_w2v50.bin", binary=True)

In [31]:
creative_id_tfidf_matrix = load_pickle(pickle_path + 'creative_id_tfidf_matrix_2')
ad_id_tfidf_matrix = load_pickle(pickle_path + 'ad_id_tfidf_matrix_2')
product_id_tfidf_matrix = load_pickle(pickle_path + 'product_id_tfidf_matrix_2')
advertiser_id_tfidf_matrix = load_pickle(pickle_path + 'advertiser_id_tfidf_matrix_2')
advertiser_id_tfidf_matrix = load_pickle(pickle_path + 'advertiser_id_tfidf_matrix_2')
click_times_tfidf_matrix = load_pickle(pickle_path + 'click_times_tfidf_matrix_2')

creative_id_tfidf_voc = load_pickle(pickle_path + 'creative_id_tfidf_voc_2')
ad_id_tfidf_voc = load_pickle(pickle_path + 'ad_id_tfidf_voc_2')
product_id_tfidf_voc = load_pickle(pickle_path + 'product_id_tfidf_voc_2')
advertiser_id_tfidf_voc = load_pickle(pickle_path + 'advertiser_id_tfidf_voc_2')
advertiser_id_tfidf_voc = load_pickle(pickle_path + 'advertiser_id_tfidf_voc_2')
click_times_tfidf_voc = load_pickle(pickle_path + 'click_times_tfidf_voc_2')



In [34]:
creative_id_idf = load_pickle(pickle_path + 'creative_id_idf')
ad_id_idf = load_pickle(pickle_path + 'ad_id_idf')
product_id_idf = load_pickle(pickle_path + 'product_id_idf')
advertiser_id_idf = load_pickle(pickle_path + 'advertiser_id_idf')
advertiser_id_idf = load_pickle(pickle_path + 'advertiser_id_idf')
click_times_idf = load_pickle(pickle_path + 'click_times_idf')

In [53]:
def embedding_tfidf_v(ll, voc, idf):
    tf = Counter(ll)
    ll = set(ll)
    id_l = []    
    for k in ll:
        if str(k) in voc:
            idf_v = math.log((1+1900000)/(1+idf[k]))+1
            id_l.append(  voc[str(k)]*tf[k]*idf_v  )
    return np.array(id_l).sum(axis=0)
creative_id_embedding_matrix = user_ids['creative_id'].map(lambda x: embedding_tfidf_v(x, creative_id_v, creative_id_idf))


In [9]:
def embedding_v(ll, voc):
    id_count = Counter(ll)
    id_l = []    
    for k in id_count.keys():
        if str(k) in voc:
            id_l.append(voc[str(k)]*id_count[k])
    return np.array(id_l).mean(axis=0)


In [ ]:
def embedding_v2(user_id, ll, voc, tfidf_matrix, tfidf_voc):
    ll = set(ll)
    id_l = []    
    for k in ll:
        if str(k) in voc and str(k) in tfidf_voc:
            id_l.append(voc[str(k)]*tfidf_matrix[user_id, tfidf_voc[str(k)]])
    return np.array(id_l).mean(axis=0)


In [15]:
embedding_matrix = user_ids['creative_id'].map(lambda x: embedding_v(x, creative_id_v))
embedding_matrix2 = np.array([embedding_matrix])[0]
save_pickle(embedding_matrix2, pickle_path + 'creative_id_embedding_matrix.pickle')

embedding_matrix = user_ids['ad_id'].map(lambda x: embedding_v(x, ad_id_v))
embedding_matrix2 = np.array([embedding_matrix])[0]
save_pickle(embedding_matrix2, pickle_path + 'ad_id_embedding_matrix.pickle')

embedding_matrix = user_ids['advertiser_id'].map(lambda x: embedding_v(x, advertiser_id_v))
embedding_matrix2 = np.array([embedding_matrix])[0]
save_pickle(embedding_matrix2, pickle_path + 'advertiser_id_embedding_matrix.pickle')


embedding_matrix = user_ids['product_id'].map(lambda x: embedding_v(x, product_id_v))
embedding_matrix2 = np.array([embedding_matrix])[0]
save_pickle(embedding_matrix2, pickle_path + 'product_id_embedding_matrix.pickle')



In [4]:
creative_id_em = pd.DataFrame(load_pickle(pickle_path + 'creative_id_embedding_matrix.pickle').astype(float))
reduce_mem_usage(creative_id_em)

ad_id_em = pd.DataFrame(load_pickle(pickle_path + 'ad_id_embedding_matrix.pickle').astype(float))
reduce_mem_usage(ad_id_em)

advertiser_id_em = pd.DataFrame(load_pickle(pickle_path + 'advertiser_id_embedding_matrix.pickle').astype(float))
reduce_mem_usage(advertiser_id_em)

product_id_em = pd.DataFrame(load_pickle(pickle_path + 'product_id_embedding_matrix.pickle').astype(float))
reduce_mem_usage(product_id_em)


Mem. usage decreased to 362.40 Mb (75.0% reduction)
Mem. usage decreased to 362.40 Mb (75.0% reduction)
Mem. usage decreased to 181.20 Mb (75.0% reduction)
Mem. usage decreased to 181.20 Mb (75.0% reduction)


In [58]:
train_filter = 700000
lr = LogisticRegression(solver='lbfgs', multi_class='ovr', n_jobs=4)
lr_model = lr.fit(embedding_matrix[0:train_filter], user_ids['gender'][0:train_filter])
lr_pred = lr_model.predict(embedding_matrix[train_filter:900000])

In [59]:
lr_pred_r = np.rint(lr_pred)
lr_pred_r = np.clip(lr_pred_r, 1, 2)
gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], lr_pred_r)
gender_accuracy,lr_pred_r.shape

(0.915715, (200000,))

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'n_estimators': 1000,
    'metric': 'mae',
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'min_child_weight': 0.01,
    'subsample_freq': 1,
    'num_leaves': 31,
    'max_depth': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'reg_alpha': 0,
    'reg_lambda': 5,
    'verbose': -1,
    'random_state': 4590,
    'n_jobs': 4,
}
logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
def lr_pred(pred_y):
    cv = True
    model_type = 'lgb'
    seed = 2020
    train_filter = 700000
    train_x = creative_id_tfidf_matrix[0:train_filter]
    train_y = user_ids[pred_y][0:train_filter]
    test_x =  creative_id_tfidf_matrix[train_filter:900000]
    if cv:
        n_fold = 2
        count_fold = 0
        preds_list = list()
        oof = np.zeros(train_x.shape[0])
        kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        kfold = kfolder.split(train_x, train_y)
        for train_index, vali_index in kfold:
            print("training......fold",count_fold)
            count_fold = count_fold + 1
            k_x_train = train_x[train_index]
            k_y_train = train_y[train_index]
            k_x_vali = train_x[vali_index]
            k_y_vali = train_y[vali_index]
            if model_type == 'lgb':
#                 reg = LogisticRegression(solver='lbfgs', multi_class='multinomial', n_jobs=-1)
#                 reg_model = reg.fit(k_x_train, k_y_train)
#                 k_pred = reg_model.predict(k_x_vali)
#                 pred = reg_model.predict(test_x)
                lgb_model = lgb.LGBMRegressor(**lgb_params)
                lgb_model = lgb_model.fit(k_x_train, k_y_train, eval_set=[(k_x_vali, k_y_vali)],
                                          early_stopping_rounds=200, verbose=100, eval_metric="mae",
                                          )
                k_pred = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration_)
                pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration_)
            preds_list.append(pred)
            oof[vali_index] = k_pred
    preds_columns = ['preds_{id}'.format(id=i) for i in range(n_fold)]
    preds_df = pd.DataFrame(data=preds_list)
    preds_df = preds_df.T
    preds_df.columns = preds_columns
    preds = list(preds_df.mean(axis=1))
    return preds, oof
#gender_pre, gender_oof = lr_pred('gender')
age_pre, age_oof = lr_pred('age')

training......fold 0
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's l1: 1.51141


KeyboardInterrupt: 

(0.920505, (200000,))  
(0.921845, (200000,))  +ad  
(0.923885, (200000,))  
(0.92392, (200000,))  all 5  


In [30]:
lr_pred_r = np.rint(gender_pre)
lr_pred_r = np.clip(lr_pred_r, 1, 2)
gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], lr_pred_r)
gender_accuracy,lr_pred_r.shape

(0.91048, (200000,))

In [56]:
def lgb_class(pred_y, num_class):    
    lgb_params = {  
        'boosting_type': 'gbdt',  
        'objective': 'multiclass',  
        'num_class': num_class,  
        'metric': 'multi_error',  
        'num_leaves': 31,  
        'learning_rate': 0.05,  
        'feature_fraction': 0.8,  
        'bagging_fraction': 0.8,  
        'bagging_freq': 5,  
        'lambda_l1': 1,  
        'lambda_l2': 5,  
        'min_gain_to_split': 0.2,  
        'verbose': -1, 
        'n_jobs':6
    }
    
    
    train_filter = 700000
    train_x = creative_id_tfidf_matrix[0:train_filter]
    train_y = (user_ids[pred_y]-1)[0:train_filter].astype(np.int)
    test_x =  creative_id_tfidf_matrix[train_filter:900000]


    n_fold = 2
    result = np.zeros((test_x.shape[0],2))
    oof = np.zeros((train_x.shape[0],2))
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2019)
    kfold = skf.split(train_x, train_y)
    count_fold = 0
    for train_index, vali_index in kfold:
        print("training......fold",count_fold)
        count_fold = count_fold + 1
        k_x_train = train_x[train_index]
        k_y_train = train_y[train_index]
        k_x_vali = train_x[vali_index]
        k_y_vali = train_y[vali_index]


        trn = lgb.Dataset(k_x_train, k_y_train)
        val = lgb.Dataset(k_x_vali, k_y_vali)
        lgb_model = lgb.train(lgb_params, train_set=trn, valid_sets=[trn, val],
                      num_boost_round=4000,early_stopping_rounds=200, verbose_eval=50)
        test_pred_proba = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration)
        val_pred_proba = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration)

        result = result + test_pred_proba
        oof[vali_index] = val_pred_proba
    return result, oof

In [57]:
age_pre, age_oof = lgb_class('age',10)


NameError: name 'StratifiedKFold' is not defined

In [71]:
gender_pre2 = gender_pre.argmax(axis=1) + 1
gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], gender_pre2)
gender_accuracy


0.928275

In [42]:
lr_pred_r = np.rint(gender_pre)
lr_pred_r = np.clip(lr_pred_r, 1, 2)
gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], lr_pred_r)
gender_accuracy,lr_pred_r.shape

(0.33007, (200000,))

0.928275